<a href="https://colab.research.google.com/github/ttb-folio/Pitchfork-Reviews/blob/main/SentimentAnalysisClassifierGPU.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers
!git clone https://github.com/ttb-folio/Pitchfork-Reviews.git

import numpy as np
import pandas as pd
import torch
from torch.utils.data import DataLoader, Dataset
from sklearn.model_selection import train_test_split
from transformers import AutoTokenizer,AutoModelForSequenceClassification, TrainingArguments, Trainer, AdamW

fatal: destination path 'Pitchfork-Reviews' already exists and is not an empty directory.


In [2]:
content_df = pd.read_csv('Pitchfork-Reviews/Data/content.csv')
review_df = pd.read_csv('Pitchfork-Reviews/Data/reviews.csv')
model_checkpoint = "distilbert-base-uncased"
text = content_df.content.fillna(' ').values
labels = review_df.score.apply(lambda x: int(x<= review_df.score.quantile(0.5))).values

tokenizer = AutoTokenizer.from_pretrained(model_checkpoint, use_fast=True)

x_train, x_test, y_train, y_test = train_test_split(text, labels, test_size=0.2, random_state=1000)

x_train = tokenizer(list(x_train), padding = True, truncation = True)
x_test = tokenizer(list(x_test),padding = True, truncation = True)

In [3]:
class customDataset(Dataset):
  def __init__(self, encodings, labels):
    self.encodings = encodings
    self.labels = labels

  def __getitem__(self,idx):
    if torch.is_tensor(idx):
      idx.tolist()
    item = {key: torch.tensor(val[idx]) for key,val in self.encodings.items()}
    item['labels'] = torch.tensor(self.labels[idx])
    return item

  def __len__(self):
    return len(self.labels)

data_train = customDataset(x_train, y_train)
data_test = customDataset(x_test, y_test)

In [4]:
batch_size = 32
num_labels = 2
model = AutoModelForSequenceClassification.from_pretrained(model_checkpoint, num_labels=num_labels)

args = TrainingArguments(
    "./train_models",
    evaluation_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=1,
    weight_decay=0.05,
    logging_dir = './logs',
    logging_steps = batch_size
)

trainer = Trainer(
    model,
    args,
    train_dataset = data_train,
    eval_dataset = data_test,
    tokenizer = tokenizer
)

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_transform.weight', 'vocab_transform.bias', 'vocab_layer_norm.weight', 'vocab_layer_norm.bias', 'vocab_projector.weight', 'vocab_projector.bias']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['pre_classifier.weight', 'pre_classifier.bias', 'classi

In [5]:
trainer.train()

Epoch,Training Loss,Validation Loss,Runtime,Samples Per Second
1,0.564900,0.574308,30.779600,119.397000


TrainOutput(global_step=460, training_loss=0.6148891158725904, metrics={'train_runtime': 442.5701, 'train_samples_per_second': 1.039, 'total_flos': 3022964066211840.0, 'epoch': 1.0, 'init_mem_cpu_alloc_delta': 1874366464, 'init_mem_gpu_alloc_delta': 268953088, 'init_mem_cpu_peaked_delta': 0, 'init_mem_gpu_peaked_delta': 0, 'train_mem_cpu_alloc_delta': 16031744, 'train_mem_gpu_alloc_delta': 816905728, 'train_mem_cpu_peaked_delta': 4096, 'train_mem_gpu_peaked_delta': 13072364544})

In [6]:
from sklearn.metrics import confusion_matrix, log_loss

outputs = trainer.predict(data_test)
print(confusion_matrix(y_test,outputs[1]))
print(log_loss(y_test, np.exp(outputs[0])))

[[1802    0]
 [   0 1873]]
0.6119545894090821
